In [8]:
# -*- coding: utf-8 -*-
# --------------------------------------------------
#
# CRF2.ipynb
#
# Token features:
#     (a) token (surface form)
#     (b) is_first           : is token at the beginning of the sentence?
#     (c) is_last            : is token at the end of the sentence?
#     (d) is_capitalized     : does token start with a capital letter? 
#     (e) is_all_capitalized : is all letters of the token capitalized?
#     (f) is_capitals_inside : is there any capitalized letter inside the token?
#     (g) is_numeric         : is token numeric?
#     (h) is_numeric_inside  : is numeric characters inside the token?
#     (i) is_alphanumeric    : is token alphanumeric?
#
# Written by cetinsamet -*- cetin.samet@metu.edu.tr
# May, 2019
# --------------------------------------------------

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from seqeval.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn_crfsuite import CRF
from tqdm import tqdm
import pickle
import scipy

In [5]:
def readFile(filepath):

    text     = []
    sentence = []

    with open(filepath, 'r') as infile:
        for line in infile:
            word, _, _, _ = line.strip().split('\t')

            if word == '<S>':
                text.append(sentence)
                sentence = []
                continue

            sentence.append(line.strip())

    return text

In [6]:
trainText = readFile('data/train.txt')
validText = readFile('data/valid.txt')
testText  = readFile('data/test.txt')

In [12]:
def getFeature(token, token_index, sentence):

    feature = {'token'             : token,
               'is_first'          : token_index == 0,
               'is_last'           : token_index == len(sentence) - 1,
               
               'is_capitalized'    : token[0].upper() == token[0],
               'is_all_capitalized': token.upper() == token,
               'is_capitals_inside': token[1:].lower() != token[1:],
               'is_numeric'        : token.isdigit(),
               'is_numeric_inside' : any([c.isdigit() for c in token]),
               'is_alphanumeric'   : token.isalnum(),
              }
    
    return feature

In [13]:
trainFeatures = []
trainLabels   = []

for sentence in tqdm(trainText):

    features = []
    labels   = []
    for i, token in enumerate(sentence):
        word, _, _, label = token.split('\t')
        
        features.append(getFeature(word, i, sentence))
        labels.append(label)
    
    trainFeatures.append(features)
    trainLabels.append(labels)

100%|██████████| 25736/25736 [00:01<00:00, 16148.51it/s]


In [16]:
validFeatures = []
validLabels   = []

for sentence in tqdm(validText):
    
    features = []
    labels   = []
    for i, token in enumerate(sentence):
        word, _, _, label = token.split('\t')
        
        features.append(getFeature(word, i, sentence))
        labels.append(label)
    
    validFeatures.append(features)
    validLabels.append(labels)

100%|██████████| 6435/6435 [00:00<00:00, 14597.69it/s]


In [17]:
testFeatures = []
testLabels   = []

for sentence in tqdm(testText):
    
    features = []
    labels   = []
    for i, token in enumerate(sentence):
        word, _, _, label = token.split('\t')
        
        features.append(getFeature(word, i, sentence))
        labels.append(label)
    
    testFeatures.append(features)
    testLabels.append(labels)

In [19]:
trainvalFeatures = trainFeatures + validFeatures
trainvalLabels   = trainLabels   + validLabels

In [20]:
# define fixed parameters and parameters to search
crf = CRF(  algorithm='lbfgs',
            max_iterations=100,
            all_possible_transitions=True,
            verbose=True)

params_space = {'c1': scipy.stats.expon(scale=0.5),
                'c2': scipy.stats.expon(scale=0.05)}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=30,
                        random_state=123,
                        scoring=f1_scorer)

rs.fit(trainvalFeatures, trainvalLabels)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/cetinsamet/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 33.5min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 77.3min finished
loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:03<00:00, 9190.46it/s] 



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 75440
Seconds required: 0.851

L-BFGS optimization
c1: 0.048313
c2: 0.028432
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.98  loss=717747.74 active=75417 feature_norm=1.00
Iter 2   time=1.56  loss=445187.31 active=75259 feature_norm=4.70
Iter 3   time=0.51  loss=370246.66 active=74430 feature_norm=4.04
Iter 4   time=1.89  loss=273160.76 active=75052 feature_norm=2.85
Iter 5   time=0.46  loss=264971.74 active=75319 feature_norm=3.37
Iter 6   time=0.46  loss=249662.99 active=75335 feature_norm=3.50
Iter 7   time=0.47  loss=222594.69 active=75312 feature_norm=5.35
Iter 8   time=0.47  loss=204100.86 active=75378 feature_norm=5.43
Iter 9   time=0.48  loss=192494.50 active=75407 feature_norm=6.25
Iter 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=None,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error...ne,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1b084f99e8>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1b084f9e10>},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn',
          scoring=make_scorer(flat_f1_score, average=weighted), verbose=1)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [34]:
crf = rs.best_estimator_
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.04831323984330459, c2=0.028431668831944812,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [ ]:
#crf.fit(trainvalFeatures, trainvalLabels)

In [35]:
# SAVE CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf2.pickle', 'wb') as outfile:
    pickle.dump(crf, outfile, pickle.HIGHEST_PROTOCOL)
    print("model is saved.")

model is saved.


In [36]:
# LOAD CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf2.pickle', 'rb') as infile:
    crf = pickle.load(infile)

In [ ]:
trainvalPredLabels = crf.predict(trainvalFeatures)

print("### TRAINVAL CLASSIFICATION REPORT ###\n")
print(classification_report(trainvalLabels, trainvalPredLabels))

In [24]:
testPredLabels  = crf.predict(testFeatures)

print("### TEST CLASSIFICATION REPORT ###\n")
print(classification_report(testLabels, testPredLabels))

### TEST CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

ORGANIZATION       0.81      0.76      0.78       862
      PERSON       0.88      0.76      0.82      1594
    LOCATION       0.82      0.83      0.82      1091
     PERCENT       1.00      0.95      0.98       107
        DATE       0.86      0.81      0.83       364
        TIME       1.00      0.35      0.52        23
       MONEY       0.86      0.73      0.79       113

   micro avg       0.85      0.79      0.82      4154
   macro avg       0.85      0.79      0.81      4154

